# Title
[]()

In [1]:

import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from silvhua import *
# from datetime import datetime
# sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\src")

In [2]:
# set the option to wrap text within cells
pd.set_option('display.max_colwidth', 100)
# pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# from `main.py`

# Batch 1

In [10]:
from functools import wraps
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\private")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from prompts import * # .py file stored in the path above
from db_orm import * 
from sources import *
from orm_summarize import *
from article_processing import *

@remote_sql_session
def get_from_queue(session, input_df, order_by='id', order='ASC'):
    """
    Return the matching records from the sources table as a pandas dataframe.

    Parameters:
    - input_df: A pandas DataFrame with the article records from the gpt_queue table or equivalent. Columns include 'title' and 'section'.
    - limit: The number of records to return.
    """
    def row_to_dict(row):
        result = session.query(Sources).filter_by(
            title=row['title'],
            section=row['section']
        ).limit(1).all()[0]
        
        sources_series = pd.Series({column.name: getattr(result, column.name) for column in result.__table__.columns})
        return sources_series

    sources_df = input_df.apply(row_to_dict, axis=1)
    ascending = True if order == 'ASC' else False
    sources_df.sort_values(order_by, ascending=ascending, inplace=True)
    return sources_df


#########
# Prep: Set parameters
folder_path = '../text/2023-08-01'
section = None
local = False
n_choices = 3
article_limit = 1
temperature = 0.7
pause_per_request=0
# summary_iteration_id = iteration_id
iteration_id = 1
chatbot_id = iteration_id
model = 'gpt-3.5-turbo-16k-0613'
# model = 'gpt-4'
save_outputs=False

def generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local, article_limit=article_limit):
    ### Set up
    qna_dict = dict()
    chatbot_dict = dict()
    references_df_dict = dict()

    # set the option to wrap text within cells
    pd.set_option('display.max_colwidth', 50)
    pd.set_option('display.max_rows', 20)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)

    ####### 
    # Step 1: Create sources table
    if local:
        text_df = parse_fulltext(folder_path, section).iloc[:article_limit if article_limit else len(text_df)]
    else:
        text_df = get_table(table='gpt_queue', limit=article_limit) # db_orm.py
    references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py

    ###### 
    # Step 2:  Add rows from gpt_queue table to sources table 
    bulk_append(table='sources', input_df=references_df_dict[iteration_id]) # db_orm.py

    # ##### 
    # Step 3: Get the new sources for summarization
    sources_df = get_from_queue(input_df=text_df, order_by='id', order='ASC')

    # ##### 
    # Step 4: Create summaries (functions contained in orm_summarize.py)
    chatbot_dict = batch_summarize( # orm_summarize.py
        sources_df, folder_path, prep_step, summarize_task, edit_task,  # parameter values found in prompts.py
        simplify_task, simplify_audience, format_task,
        chatbot_dict, temperature=temperature,
        system_role=system_role, model=model, max_tokens=1000,
        n_choices=n_choices, pause_per_request=pause_per_request,
        iteration_id=iteration_id, save_outputs=save_outputs
        )
    #########
    # Step 5: Create summaries table
    qna_dict = create_summaries_df(
        qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id
        )

    ##########
    # Step 5: Add results to summaries and prompts table 
    bulk_append(table='summaries', input_df=qna_dict[iteration_id]) # db_orm.py

    return qna_dict[iteration_id]

if __name__ == "__main__":
    qna_dict = generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local=True, article_limit=article_limit)
    # print(qna_dict[iteration_id)
    qna_dict



Keys for text_dict: dict_keys([1])

Adding 1 rows to the database...
	** Already exists in the database: Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women.
New records added successfully (if applicable)!
**Text #5 prompt #1 of 4**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo-16k-0613
Chaining class instance created
	Done creating prompt
	Sending request to gpt-3.5-turbo-16k-0613
		Requesting 3 choices using gpt-3.5-turbo-16k-0613
	Done sending request to GPT-3
	...Completed
**Text #5 prompt #2 of 4**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo-16k-0613
Chaining class instance created
	Done creating prompt
	Sending request to gpt-3.5-turbo-16k-0613
		Requesting 3 choices using gpt-3.5-turbo-16k-0613
	Done sending request to GPT-3
	...Completed
**Text #5 prompt #3 of 4**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo-16k-0613
Chaining class instance created
	Done creating prompt
	Sending request to g

In [11]:
qna_dict

,timestamp,reference_id,article_title,choice,text,system_role,model,temperature,prep_step,summarize_task,edit_task,simplify_task,simplify_audience,format_task,full_summarize_task,folder,summary,headline,simple_summary
0,2023-08-01 15:12:13.534524-07:00,5,Hypohydration but not Menstrual Phase Influenc...,1,Pain is recognized as a public health problem ...,You are someone who loves to read health resea...,gpt-3.5-turbo-16k-0613,0.7,"In the summary, cover the following informatio...",Write a casual text message to your friend abo...,Once you have written your text message: \...,3. Rewrite the text in a fun tone. Follow t...,people who like fun facts but don't know much ...,4. Return your final response in a JSON format...,Write a casual text message to your friend abo...,text/2023-08-01,A recent study found that mild dehydration can...,New research shows that not drinking enough wa...,Did you know that not drinking enough water ca...
1,2023-08-01 15:12:13.534524-07:00,5,Hypohydration but not Menstrual Phase Influenc...,2,Pain is recognized as a public health problem ...,You are someone who loves to read health resea...,gpt-3.5-turbo-16k-0613,0.7,"In the summary, cover the following informatio...",Write a casual text message to your friend abo...,Once you have written your text message: \...,3. Rewrite the text in a fun tone. Follow t...,people who like fun facts but don't know much ...,4. Return your final response in a JSON format...,Write a casual text message to your friend abo...,text/2023-08-01,A recent study found that mild dehydration can...,Did you know: Dehydration can increase pain se...,Hey! Just read this interesting study that fou...
2,2023-08-01 15:12:13.534524-07:00,5,Hypohydration but not Menstrual Phase Influenc...,3,Pain is recognized as a public health problem ...,You are someone who loves to read health resea...,gpt-3.5-turbo-16k-0613,0.7,"In the summary, cover the following informatio...",Write a casual text message to your friend abo...,Once you have written your text message: \...,3. Rewrite the text in a fun tone. Follow t...,people who like fun facts but don't know much ...,4. Return your final response in a JSON format...,Write a casual text message to your friend abo...,text/2023-08-01,A recent study found that mild dehydration can...,Discover how dehydration can affect pain sensi...,Hey! Did you know that dehydration can actuall...
3,2023-08-01 15:12:18.518566-07:00,5,Hypohydration but not Menstrual Phase Influenc...,1,Pain is recognized as a public health problem ...,You are someone who loves to read health resea...,gpt-3.5-turbo-16k-0613,0.7,"In the summary, cover the following informatio...",Write a casual text message to your friend abo...,Once you have written your text message: \...,3. Rewrite the text in a fun tone. Follow t...,a lay audience,4. Return your final response in a JSON format...,Write a casual text message to your friend abo...,text/2023-08-01,"According to a recent study, mild dehydration ...",New research shows that dehydration can increa...,Hey! Just read this fascinating study that sho...
4,2023-08-01 15:12:18.518566-07:00,5,Hypohydration but not Menstrual Phase Influenc...,2,Pain is recognized as a public health problem ...,You are someone who loves to read health resea...,gpt-3.5-turbo-16k-0613,0.7,"In the summary, cover the following informatio...",Write a casual text message to your friend abo...,Once you have written your text message: \...,3. Rewrite the text in a fun tone. Follow t...,a lay audience,4. Return your final response in a JSON format...,Write a casual text message to your friend abo...,text/2023-08-01,Hey! I just read some fascinating research abo...,New Research Shows How Hydration Affects Pain ...,Hey! I just read this super cool study about h...
5,2023-08-01 15:12:18.518566-07:00,5,Hypohydration but not Menstrual Phase Influenc...,3,Pain is recognized as a public health problem ...,You are someone who loves to read health resea...,gpt-3.5-turbo-16k-0613,0.7

# batch 2 update prompts 15:21

In [12]:
from functools import wraps
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\private")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from prompts import * # .py file stored in the path above
from db_orm import * 
from sources import *
from orm_summarize import *
from article_processing import *

@remote_sql_session
def get_from_queue(session, input_df, order_by='id', order='ASC'):
    """
    Return the matching records from the sources table as a pandas dataframe.

    Parameters:
    - input_df: A pandas DataFrame with the article records from the gpt_queue table or equivalent. Columns include 'title' and 'section'.
    - limit: The number of records to return.
    """
    def row_to_dict(row):
        result = session.query(Sources).filter_by(
            title=row['title'],
            section=row['section']
        ).limit(1).all()[0]
        
        sources_series = pd.Series({column.name: getattr(result, column.name) for column in result.__table__.columns})
        return sources_series

    sources_df = input_df.apply(row_to_dict, axis=1)
    ascending = True if order == 'ASC' else False
    sources_df.sort_values(order_by, ascending=ascending, inplace=True)
    return sources_df


#########
# Prep: Set parameters
folder_path = '../text/2023-08-01'
section = None
local = False
n_choices = 3
article_limit = 1
temperature = 0.7
pause_per_request=0
# summary_iteration_id = iteration_id
iteration_id = 1
chatbot_id = iteration_id
model = 'gpt-3.5-turbo-16k-0613'
# model = 'gpt-4'
save_outputs=False

def generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local, article_limit=article_limit):
    ### Set up
    qna_dict = dict()
    chatbot_dict = dict()
    references_df_dict = dict()

    # set the option to wrap text within cells
    pd.set_option('display.max_colwidth', 50)
    pd.set_option('display.max_rows', 20)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)

    ####### 
    # Step 1: Create sources table
    if local:
        text_df = parse_fulltext(folder_path, section).iloc[:article_limit if article_limit else len(text_df)]
    else:
        text_df = get_table(table='gpt_queue', limit=article_limit) # db_orm.py
    references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py

    ###### 
    # Step 2:  Add rows from gpt_queue table to sources table 
    bulk_append(table='sources', input_df=references_df_dict[iteration_id]) # db_orm.py

    # ##### 
    # Step 3: Get the new sources for summarization
    sources_df = get_from_queue(input_df=text_df, order_by='id', order='ASC')

    # ##### 
    # Step 4: Create summaries (functions contained in orm_summarize.py)
    chatbot_dict = batch_summarize( # orm_summarize.py
        sources_df, folder_path, prep_step, summarize_task, edit_task,  # parameter values found in prompts.py
        simplify_task, simplify_audience, format_task,
        chatbot_dict, temperature=temperature,
        system_role=system_role, model=model, max_tokens=1000,
        n_choices=n_choices, pause_per_request=pause_per_request,
        iteration_id=iteration_id, save_outputs=save_outputs
        )
    #########
    # Step 5: Create summaries table
    qna_dict = create_summaries_df(
        qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id
        )

    ##########
    # Step 5: Add results to summaries and prompts table 
    bulk_append(table='summaries', input_df=qna_dict[iteration_id]) # db_orm.py

    return qna_dict[iteration_id]

if __name__ == "__main__":
    qna_dict = generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local=True, article_limit=article_limit)
    # print(qna_dict[iteration_id)
    qna_dict



Keys for text_dict: dict_keys([1])

Adding 1 rows to the database...
	** Already exists in the database: Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women.
New records added successfully (if applicable)!
**Text #5 prompt #1 of 4**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo-16k-0613
Chaining class instance created
	Done creating prompt
	Sending request to gpt-3.5-turbo-16k-0613
		Requesting 3 choices using gpt-3.5-turbo-16k-0613
	Done sending request to GPT-3
	...Completed
**Text #5 prompt #2 of 4**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo-16k-0613
Chaining class instance created
	Done creating prompt
	Sending request to gpt-3.5-turbo-16k-0613
		Requesting 3 choices using gpt-3.5-turbo-16k-0613
	Done sending request to GPT-3
	...Completed
**Text #5 prompt #3 of 4**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo-16k-0613
Chaining class instance created
	Done creating prompt
	Sending request to g

## Repeat after refreshing kernel

In [1]:
from functools import wraps
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\private")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from prompts import * # .py file stored in the path above
from db_orm import * 
from sources import *
from orm_summarize import *
from article_processing import *

@remote_sql_session
def get_from_queue(session, input_df, order_by='id', order='ASC'):
    """
    Return the matching records from the sources table as a pandas dataframe.

    Parameters:
    - input_df: A pandas DataFrame with the article records from the gpt_queue table or equivalent. Columns include 'title' and 'section'.
    - limit: The number of records to return.
    """
    def row_to_dict(row):
        result = session.query(Sources).filter_by(
            title=row['title'],
            section=row['section']
        ).limit(1).all()[0]
        
        sources_series = pd.Series({column.name: getattr(result, column.name) for column in result.__table__.columns})
        return sources_series

    sources_df = input_df.apply(row_to_dict, axis=1)
    ascending = True if order == 'ASC' else False
    sources_df.sort_values(order_by, ascending=ascending, inplace=True)
    return sources_df


#########
# Prep: Set parameters
folder_path = '../text/2023-08-01'
section = None
local = False
n_choices = 3
article_limit = 1
temperature = 0.7
pause_per_request=0
# summary_iteration_id = iteration_id
iteration_id = 1
chatbot_id = iteration_id
model = 'gpt-3.5-turbo-16k-0613'
# model = 'gpt-4'
save_outputs=False

def generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local, article_limit=article_limit):
    ### Set up
    qna_dict = dict()
    chatbot_dict = dict()
    references_df_dict = dict()

    # set the option to wrap text within cells
    pd.set_option('display.max_colwidth', 50)
    pd.set_option('display.max_rows', 20)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)

    ####### 
    # Step 1: Create sources table
    if local:
        text_df = parse_fulltext(folder_path, section).iloc[:article_limit if article_limit else len(text_df)]
    else:
        text_df = get_table(table='gpt_queue', limit=article_limit) # db_orm.py
    references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py

    ###### 
    # Step 2:  Add rows from gpt_queue table to sources table 
    bulk_append(table='sources', input_df=references_df_dict[iteration_id]) # db_orm.py

    # ##### 
    # Step 3: Get the new sources for summarization
    sources_df = get_from_queue(input_df=text_df, order_by='id', order='ASC')

    # ##### 
    # Step 4: Create summaries (functions contained in orm_summarize.py)
    chatbot_dict = batch_summarize( # orm_summarize.py
        sources_df, folder_path, prep_step, summarize_task, edit_task,  # parameter values found in prompts.py
        simplify_task, simplify_audience, format_task,
        chatbot_dict, temperature=temperature,
        system_role=system_role, model=model, max_tokens=1000,
        n_choices=n_choices, pause_per_request=pause_per_request,
        iteration_id=iteration_id, save_outputs=save_outputs
        )
    #########
    # Step 5: Create summaries table
    qna_dict = create_summaries_df(
        qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id
        )

    ##########
    # Step 5: Add results to summaries and prompts table 
    bulk_append(table='summaries', input_df=qna_dict[iteration_id]) # db_orm.py

    return qna_dict[iteration_id]

if __name__ == "__main__":
    qna_dict = generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local=True, article_limit=article_limit)
    # print(qna_dict[iteration_id)
    qna_dict



Keys for text_dict: dict_keys([1])

Adding 1 rows to the database...
	** Already exists in the database: Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women.
New records added successfully (if applicable)!
**Text #5 prompt #1 of 1**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo-16k-0613
Chaining class instance created
	Done creating prompt
	Sending request to gpt-3.5-turbo-16k-0613
		Requesting 3 choices using gpt-3.5-turbo-16k-0613
	Done sending request to GPT-3
	...Completed
Processing 5_prompt00...
Original summaries DataFrame shape: (3, 19)
	Original summaries Dataframe columns: Index(['choice', 'timestamp', 'reference_id', 'article_title', 'text',
       'system_role', 'model', 'temperature', 'prep_step', 'summarize_task',
       'edit_task', 'simplify_task', 'simplify_audience', 'format_task',
       'full_summarize_task', 'folder', 'summary', 'headline',
       'simple_summary'],
      dtype='object')
Adding 3 rows to the database

# Batch 3 discussion only

In [2]:
from functools import wraps
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\private")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from prompts import * # .py file stored in the path above
from db_orm import * 
from sources import *
from orm_summarize import *
from article_processing import *

@remote_sql_session
def get_from_queue(session, input_df, order_by='id', order='ASC'):
    """
    Return the matching records from the sources table as a pandas dataframe.

    Parameters:
    - input_df: A pandas DataFrame with the article records from the gpt_queue table or equivalent. Columns include 'title' and 'section'.
    - limit: The number of records to return.
    """
    def row_to_dict(row):
        result = session.query(Sources).filter_by(
            title=row['title'],
            section=row['section']
        ).limit(1).all()[0]
        
        sources_series = pd.Series({column.name: getattr(result, column.name) for column in result.__table__.columns})
        return sources_series

    sources_df = input_df.apply(row_to_dict, axis=1)
    ascending = True if order == 'ASC' else False
    sources_df.sort_values(order_by, ascending=ascending, inplace=True)
    return sources_df


#########
# Prep: Set parameters
folder_path = '../text/2023-08-01 discussion'
section = None
local = False
n_choices = 3
article_limit = 1
temperature = 0.7
pause_per_request=0
# summary_iteration_id = iteration_id
iteration_id = 1
chatbot_id = iteration_id
model = 'gpt-3.5-turbo-16k-0613'
# model = 'gpt-4'
save_outputs=False

def generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local, article_limit=article_limit):
    ### Set up
    qna_dict = dict()
    chatbot_dict = dict()
    references_df_dict = dict()

    # set the option to wrap text within cells
    pd.set_option('display.max_colwidth', 50)
    pd.set_option('display.max_rows', 20)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)

    ####### 
    # Step 1: Create sources table
    if local:
        text_df = parse_fulltext(folder_path, section).iloc[:article_limit if article_limit else len(text_df)]
    else:
        text_df = get_table(table='gpt_queue', limit=article_limit) # db_orm.py
    references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py

    ###### 
    # Step 2:  Add rows from gpt_queue table to sources table 
    bulk_append(table='sources', input_df=references_df_dict[iteration_id]) # db_orm.py

    # ##### 
    # Step 3: Get the new sources for summarization
    sources_df = get_from_queue(input_df=text_df, order_by='id', order='ASC')

    # ##### 
    # Step 4: Create summaries (functions contained in orm_summarize.py)
    chatbot_dict = batch_summarize( # orm_summarize.py
        sources_df, folder_path, prep_step, summarize_task, edit_task,  # parameter values found in prompts.py
        simplify_task, simplify_audience, format_task,
        chatbot_dict, temperature=temperature,
        system_role=system_role, model=model, max_tokens=1000,
        n_choices=n_choices, pause_per_request=pause_per_request,
        iteration_id=iteration_id, save_outputs=save_outputs
        )
    #########
    # Step 5: Create summaries table
    qna_dict = create_summaries_df(
        qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id
        )

    ##########
    # Step 5: Add results to summaries and prompts table 
    bulk_append(table='summaries', input_df=qna_dict[iteration_id]) # db_orm.py

    return qna_dict[iteration_id]

if __name__ == "__main__":
    qna_dict = generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local=True, article_limit=article_limit)
    # print(qna_dict[iteration_id)
    qna_dict



Keys for text_dict: dict_keys([1])

Adding 1 rows to the database...
	** Already exists in the database: Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women.
New records added successfully (if applicable)!
**Text #5 prompt #1 of 1**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo-16k-0613
Chaining class instance created
	Done creating prompt
	Sending request to gpt-3.5-turbo-16k-0613
		Requesting 3 choices using gpt-3.5-turbo-16k-0613
	Done sending request to GPT-3
	...Completed
Processing 5_prompt00...
Original summaries DataFrame shape: (3, 19)
	Original summaries Dataframe columns: Index(['choice', 'timestamp', 'reference_id', 'article_title', 'text',
       'system_role', 'model', 'temperature', 'prep_step', 'summarize_task',
       'edit_task', 'simplify_task', 'simplify_audience', 'format_task',
       'full_summarize_task', 'folder', 'summary', 'headline',
       'simple_summary'],
      dtype='object')
Adding 3 rows to the database

# Batch 4 use gpt-3.5-turbo

In [3]:
from functools import wraps
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\private")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from prompts import * # .py file stored in the path above
from db_orm import * 
from sources import *
from orm_summarize import *
from article_processing import *

@remote_sql_session
def get_from_queue(session, input_df, order_by='id', order='ASC'):
    """
    Return the matching records from the sources table as a pandas dataframe.

    Parameters:
    - input_df: A pandas DataFrame with the article records from the gpt_queue table or equivalent. Columns include 'title' and 'section'.
    - limit: The number of records to return.
    """
    def row_to_dict(row):
        result = session.query(Sources).filter_by(
            title=row['title'],
            section=row['section']
        ).limit(1).all()[0]
        
        sources_series = pd.Series({column.name: getattr(result, column.name) for column in result.__table__.columns})
        return sources_series

    sources_df = input_df.apply(row_to_dict, axis=1)
    ascending = True if order == 'ASC' else False
    sources_df.sort_values(order_by, ascending=ascending, inplace=True)
    return sources_df


#########
# Prep: Set parameters
folder_path = '../text/2023-08-01 discussion'
section = None
local = False
n_choices = 2
article_limit = 1
temperature = 0.7
pause_per_request=0
# summary_iteration_id = iteration_id
iteration_id = 1
chatbot_id = iteration_id
# model = 'gpt-3.5-turbo-16k-0613'
model = 'gpt-3.5-turbo'

# model = 'gpt-4'
save_outputs=False

def generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local, article_limit=article_limit):
    ### Set up
    qna_dict = dict()
    chatbot_dict = dict()
    references_df_dict = dict()

    # set the option to wrap text within cells
    pd.set_option('display.max_colwidth', 50)
    pd.set_option('display.max_rows', 20)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)

    ####### 
    # Step 1: Create sources table
    if local:
        text_df = parse_fulltext(folder_path, section).iloc[:article_limit if article_limit else len(text_df)]
    else:
        text_df = get_table(table='gpt_queue', limit=article_limit) # db_orm.py
    references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py

    ###### 
    # Step 2:  Add rows from gpt_queue table to sources table 
    bulk_append(table='sources', input_df=references_df_dict[iteration_id]) # db_orm.py

    # ##### 
    # Step 3: Get the new sources for summarization
    sources_df = get_from_queue(input_df=text_df, order_by='id', order='ASC')

    # ##### 
    # Step 4: Create summaries (functions contained in orm_summarize.py)
    chatbot_dict = batch_summarize( # orm_summarize.py
        sources_df, folder_path, prep_step, summarize_task, edit_task,  # parameter values found in prompts.py
        simplify_task, simplify_audience, format_task,
        chatbot_dict, temperature=temperature,
        system_role=system_role, model=model, max_tokens=1000,
        n_choices=n_choices, pause_per_request=pause_per_request,
        iteration_id=iteration_id, save_outputs=save_outputs
        )
    #########
    # Step 5: Create summaries table
    qna_dict = create_summaries_df(
        qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id
        )

    ##########
    # Step 5: Add results to summaries and prompts table 
    bulk_append(table='summaries', input_df=qna_dict[iteration_id]) # db_orm.py

    return qna_dict[iteration_id]

if __name__ == "__main__":
    qna_dict = generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local=True, article_limit=article_limit)
    # print(qna_dict[iteration_id)
    qna_dict



Keys for text_dict: dict_keys([1])

Adding 1 rows to the database...
	** Already exists in the database: Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women.
New records added successfully (if applicable)!
**Text #5 prompt #1 of 1**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo
Chaining class instance created
	Done creating prompt
	Sending request to gpt-3.5-turbo
		Requesting 2 choices using gpt-3.5-turbo
An error occurred on line 89 in C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src\orm_summarize.py : This model's maximum context length is 4097 tokens. However, your messages resulted in 9075 tokens. Please reduce the length of the messages.
	**API request failed for `.summarize()`**
	...Completed
Processing 5_prompt00...
	An error occurred on line 195 in C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src\orm_summarize.py: 'summary'
Error creating DataFrame from 5_prompt00: 'summary'


ValueError: No objects to concatenate

## try with GPT-4

In [5]:
from functools import wraps
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\private")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from prompts import * # .py file stored in the path above
from db_orm import * 
from sources import *
from orm_summarize import *
from article_processing import *

@remote_sql_session
def get_from_queue(session, input_df, order_by='id', order='ASC'):
    """
    Return the matching records from the sources table as a pandas dataframe.

    Parameters:
    - input_df: A pandas DataFrame with the article records from the gpt_queue table or equivalent. Columns include 'title' and 'section'.
    - limit: The number of records to return.
    """
    def row_to_dict(row):
        result = session.query(Sources).filter_by(
            title=row['title'],
            section=row['section']
        ).limit(1).all()[0]
        
        sources_series = pd.Series({column.name: getattr(result, column.name) for column in result.__table__.columns})
        return sources_series

    sources_df = input_df.apply(row_to_dict, axis=1)
    ascending = True if order == 'ASC' else False
    sources_df.sort_values(order_by, ascending=ascending, inplace=True)
    return sources_df


#########
# Prep: Set parameters
folder_path = '../text/2023-08-01 discussion'
section = None
local = False
n_choices = 2
article_limit = 1
temperature = 0.7
pause_per_request=0
# summary_iteration_id = iteration_id
iteration_id = 1
chatbot_id = iteration_id
# model = 'gpt-3.5-turbo-16k-0613'
# model = 'gpt-3.5-turbo'

model = 'gpt-4'
save_outputs=False

def generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local, article_limit=article_limit):
    ### Set up
    qna_dict = dict()
    chatbot_dict = dict()
    references_df_dict = dict()

    # set the option to wrap text within cells
    pd.set_option('display.max_colwidth', 50)
    pd.set_option('display.max_rows', 20)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)

    ####### 
    # Step 1: Create sources table
    if local:
        text_df = parse_fulltext(folder_path, section).iloc[:article_limit if article_limit else len(text_df)]
    else:
        text_df = get_table(table='gpt_queue', limit=article_limit) # db_orm.py
    references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py

    ###### 
    # Step 2:  Add rows from gpt_queue table to sources table 
    bulk_append(table='sources', input_df=references_df_dict[iteration_id]) # db_orm.py

    # ##### 
    # Step 3: Get the new sources for summarization
    sources_df = get_from_queue(input_df=text_df, order_by='id', order='ASC')

    # ##### 
    # Step 4: Create summaries (functions contained in orm_summarize.py)
    chatbot_dict = batch_summarize( # orm_summarize.py
        sources_df, folder_path, prep_step, summarize_task, edit_task,  # parameter values found in prompts.py
        simplify_task, simplify_audience, format_task,
        chatbot_dict, temperature=temperature,
        system_role=system_role, model=model, max_tokens=1000,
        n_choices=n_choices, pause_per_request=pause_per_request,
        iteration_id=iteration_id, save_outputs=save_outputs
        )
    #########
    # Step 5: Create summaries table
    qna_dict = create_summaries_df(
        qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id
        )

    ##########
    # Step 5: Add results to summaries and prompts table 
    bulk_append(table='summaries', input_df=qna_dict[iteration_id]) # db_orm.py

    return qna_dict[iteration_id]

if __name__ == "__main__":
    qna_dict = generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local=True, article_limit=article_limit)
    # print(qna_dict[iteration_id)
    qna_dict



Keys for text_dict: dict_keys([1])

Adding 1 rows to the database...
	** Already exists in the database: Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women.
New records added successfully (if applicable)!
**Text #5 prompt #1 of 1**
Creating Chaining class instance
***OpenAI model: gpt-4
Chaining class instance created
	Done creating prompt
	Sending request to gpt-4
		Requesting 2 choices using gpt-4
An error occurred on line 89 in C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src\orm_summarize.py : Rate limit reached for 10KTPM-200RPM in organization org-4l8HUKDtXhH0T7iFErf1JSJg on tokens per min. Limit: 10000 / min. Please try again in 6ms. Contact us through our help center at help.openai.com if you continue to have issues.
	**API request failed for `.summarize()`**
	...Completed
Processing 5_prompt00...
	An error occurred on line 195 in C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src\orm_summarize.py

ValueError: No objects to concatenate

# batch 5 reduce prompt length

In [2]:
from functools import wraps
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\private")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from prompts import * # .py file stored in the path above
from db_orm import * 
from sources import *
from orm_summarize import *
from article_processing import *

@remote_sql_session
def get_from_queue(session, input_df, order_by='id', order='ASC'):
    """
    Return the matching records from the sources table as a pandas dataframe.

    Parameters:
    - input_df: A pandas DataFrame with the article records from the gpt_queue table or equivalent. Columns include 'title' and 'section'.
    - limit: The number of records to return.
    """
    def row_to_dict(row):
        result = session.query(Sources).filter_by(
            title=row['title'],
            section=row['section']
        ).limit(1).all()[0]
        
        sources_series = pd.Series({column.name: getattr(result, column.name) for column in result.__table__.columns})
        return sources_series

    sources_df = input_df.apply(row_to_dict, axis=1)
    ascending = True if order == 'ASC' else False
    sources_df.sort_values(order_by, ascending=ascending, inplace=True)
    return sources_df


#########
# Prep: Set parameters
folder_path = '../text/2023-08-01 discussion'
section = 'discussion'
local = False
n_choices = 2
article_limit = 1
temperature = 0.7
pause_per_request=0
# summary_iteration_id = iteration_id
iteration_id = 1
chatbot_id = iteration_id
# model = 'gpt-3.5-turbo-16k-0613'
model = 'gpt-3.5-turbo'

# model = 'gpt-4'
save_outputs=False

def generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local, article_limit=article_limit):
    ### Set up
    qna_dict = dict()
    chatbot_dict = dict()
    references_df_dict = dict()

    # set the option to wrap text within cells
    pd.set_option('display.max_colwidth', 50)
    pd.set_option('display.max_rows', 20)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)

    ####### 
    # Step 1: Create sources table
    if local:
        text_df = parse_fulltext(folder_path, section).iloc[:article_limit if article_limit else len(text_df)]
    else:
        text_df = get_table(table='gpt_queue', limit=article_limit) # db_orm.py
    references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py

    ###### 
    # Step 2:  Add rows from gpt_queue table to sources table 
    bulk_append(table='sources', input_df=references_df_dict[iteration_id]) # db_orm.py

    # ##### 
    # Step 3: Get the new sources for summarization
    sources_df = get_from_queue(input_df=text_df, order_by='id', order='ASC')

    # ##### 
    # Step 4: Create summaries (functions contained in orm_summarize.py)
    chatbot_dict = batch_summarize( # orm_summarize.py
        sources_df, folder_path, prep_step, summarize_task, edit_task,  # parameter values found in prompts.py
        simplify_task, simplify_audience, format_task,
        chatbot_dict, temperature=temperature,
        system_role=system_role, model=model, max_tokens=800,
        n_choices=n_choices, pause_per_request=pause_per_request,
        iteration_id=iteration_id, save_outputs=save_outputs
        )
    #########
    # Step 5: Create summaries table
    qna_dict = create_summaries_df(
        qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id
        )

    ##########
    # Step 5: Add results to summaries and prompts table 
    bulk_append(table='summaries', input_df=qna_dict[iteration_id]) # db_orm.py

    return qna_dict[iteration_id]

if __name__ == "__main__":
    qna_dict = generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local=True, article_limit=article_limit)
    # print(qna_dict[iteration_id)
    qna_dict



Keys for text_dict: dict_keys([1])

Adding 1 rows to the database...
	** Already exists in the database: Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women.
New records added successfully (if applicable)!
**Text #16 prompt #1 of 1**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo
Chaining class instance created
	Done creating prompt
	Sending request to gpt-3.5-turbo
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Completed
Processing 16_prompt00...
Original summaries DataFrame shape: (2, 19)
	Original summaries Dataframe columns: Index(['choice', 'timestamp', 'reference_id', 'article_title', 'text',
       'system_role', 'model', 'temperature', 'prep_step', 'summarize_task',
       'edit_task', 'simplify_task', 'simplify_audience', 'format_task',
       'full_summarize_task', 'folder', 'summary', 'headline',
       'simple_summary'],
      dtype='object')
Adding 2 rows to the database...
	Reference #16: Effec

## max_tokens = 1000

In [3]:
from functools import wraps
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\private")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from prompts import * # .py file stored in the path above
from db_orm import * 
from sources import *
from orm_summarize import *
from article_processing import *

@remote_sql_session
def get_from_queue(session, input_df, order_by='id', order='ASC'):
    """
    Return the matching records from the sources table as a pandas dataframe.

    Parameters:
    - input_df: A pandas DataFrame with the article records from the gpt_queue table or equivalent. Columns include 'title' and 'section'.
    - limit: The number of records to return.
    """
    def row_to_dict(row):
        result = session.query(Sources).filter_by(
            title=row['title'],
            section=row['section']
        ).limit(1).all()[0]
        
        sources_series = pd.Series({column.name: getattr(result, column.name) for column in result.__table__.columns})
        return sources_series

    sources_df = input_df.apply(row_to_dict, axis=1)
    ascending = True if order == 'ASC' else False
    sources_df.sort_values(order_by, ascending=ascending, inplace=True)
    return sources_df


#########
# Prep: Set parameters
folder_path = '../text/2023-08-01 discussion'
section = 'discussion'
local = False
n_choices = 3
article_limit = 1
temperature = 0.7
pause_per_request=0
# summary_iteration_id = iteration_id
iteration_id = 1
chatbot_id = iteration_id
# model = 'gpt-3.5-turbo-16k-0613'
model = 'gpt-3.5-turbo'

# model = 'gpt-4'
save_outputs=False

def generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local, article_limit=article_limit):
    ### Set up
    qna_dict = dict()
    chatbot_dict = dict()
    references_df_dict = dict()

    # set the option to wrap text within cells
    pd.set_option('display.max_colwidth', 50)
    pd.set_option('display.max_rows', 20)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)

    ####### 
    # Step 1: Create sources table
    if local:
        text_df = parse_fulltext(folder_path, section).iloc[:article_limit if article_limit else len(text_df)]
    else:
        text_df = get_table(table='gpt_queue', limit=article_limit) # db_orm.py
    references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py

    ###### 
    # Step 2:  Add rows from gpt_queue table to sources table 
    bulk_append(table='sources', input_df=references_df_dict[iteration_id]) # db_orm.py

    # ##### 
    # Step 3: Get the new sources for summarization
    sources_df = get_from_queue(input_df=text_df, order_by='id', order='ASC')

    # ##### 
    # Step 4: Create summaries (functions contained in orm_summarize.py)
    chatbot_dict = batch_summarize( # orm_summarize.py
        sources_df, folder_path, prep_step, summarize_task, edit_task,  # parameter values found in prompts.py
        simplify_task, simplify_audience, format_task,
        chatbot_dict, temperature=temperature,
        system_role=system_role, model=model, max_tokens=1000,
        n_choices=n_choices, pause_per_request=pause_per_request,
        iteration_id=iteration_id, save_outputs=save_outputs
        )
    #########
    # Step 5: Create summaries table
    qna_dict = create_summaries_df(
        qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id
        )

    ##########
    # Step 5: Add results to summaries and prompts table 
    bulk_append(table='summaries', input_df=qna_dict[iteration_id]) # db_orm.py

    return qna_dict[iteration_id]

if __name__ == "__main__":
    qna_dict = generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local=True, article_limit=article_limit)
    # print(qna_dict[iteration_id)
    qna_dict



Keys for text_dict: dict_keys([1])

Adding 1 rows to the database...
	** Already exists in the database: Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women.
New records added successfully (if applicable)!
**Text #16 prompt #1 of 1**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo
Chaining class instance created
	Done creating prompt
	Sending request to gpt-3.5-turbo
		Requesting 3 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Completed
Processing 16_prompt00...
Original summaries DataFrame shape: (3, 19)
	Original summaries Dataframe columns: Index(['choice', 'timestamp', 'reference_id', 'article_title', 'text',
       'system_role', 'model', 'temperature', 'prep_step', 'summarize_task',
       'edit_task', 'simplify_task', 'simplify_audience', 'format_task',
       'full_summarize_task', 'folder', 'summary', 'headline',
       'simple_summary'],
      dtype='object')
Adding 3 rows to the database...
	Reference #16: Effec

# batch 6: add back system role

In [1]:
from functools import wraps
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\private")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from prompts import * # .py file stored in the path above
from db_orm import * 
from sources import *
from orm_summarize import *
from article_processing import *

@remote_sql_session
def get_from_queue(session, input_df, order_by='id', order='ASC'):
    """
    Return the matching records from the sources table as a pandas dataframe.

    Parameters:
    - input_df: A pandas DataFrame with the article records from the gpt_queue table or equivalent. Columns include 'title' and 'section'.
    - limit: The number of records to return.
    """
    def row_to_dict(row):
        result = session.query(Sources).filter_by(
            title=row['title'],
            section=row['section']
        ).limit(1).all()[0]
        
        sources_series = pd.Series({column.name: getattr(result, column.name) for column in result.__table__.columns})
        return sources_series

    sources_df = input_df.apply(row_to_dict, axis=1)
    ascending = True if order == 'ASC' else False
    sources_df.sort_values(order_by, ascending=ascending, inplace=True)
    return sources_df


#########
# Prep: Set parameters
folder_path = '../text/2023-08-01 discussion'
section = 'discussion'
local = False
n_choices = 2
article_limit = 1
temperature = 0.7
pause_per_request=0
# summary_iteration_id = iteration_id
iteration_id = 1
chatbot_id = iteration_id
# model = 'gpt-3.5-turbo-16k-0613'
model = 'gpt-3.5-turbo'

# model = 'gpt-4'
save_outputs=False

def generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local, article_limit=article_limit):
    ### Set up
    qna_dict = dict()
    chatbot_dict = dict()
    references_df_dict = dict()

    # set the option to wrap text within cells
    pd.set_option('display.max_colwidth', 50)
    pd.set_option('display.max_rows', 20)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)

    ####### 
    # Step 1: Create sources table
    if local:
        text_df = parse_fulltext(folder_path, section).iloc[:article_limit if article_limit else len(text_df)]
    else:
        text_df = get_table(table='gpt_queue', limit=article_limit) # db_orm.py
    references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py

    ###### 
    # Step 2:  Add rows from gpt_queue table to sources table 
    bulk_append(table='sources', input_df=references_df_dict[iteration_id]) # db_orm.py

    # ##### 
    # Step 3: Get the new sources for summarization
    sources_df = get_from_queue(input_df=text_df, order_by='id', order='ASC')

    # ##### 
    # Step 4: Create summaries (functions contained in orm_summarize.py)
    chatbot_dict = batch_summarize( # orm_summarize.py
        sources_df, folder_path, prep_step, summarize_task, edit_task,  # parameter values found in prompts.py
        simplify_task, simplify_audience, format_task,
        chatbot_dict, temperature=temperature,
        system_role=system_role, model=model, max_tokens=800,
        n_choices=n_choices, pause_per_request=pause_per_request,
        iteration_id=iteration_id, save_outputs=save_outputs
        )
    #########
    # Step 5: Create summaries table
    qna_dict = create_summaries_df(
        qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id
        )

    ##########
    # Step 5: Add results to summaries and prompts table 
    bulk_append(table='summaries', input_df=qna_dict[iteration_id]) # db_orm.py

    return qna_dict[iteration_id]

if __name__ == "__main__":
    qna_dict = generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local=True, article_limit=article_limit)
    # print(qna_dict[iteration_id)
    qna_dict



Keys for text_dict: dict_keys([1])

Adding 1 rows to the database...
	** Already exists in the database: Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women.
New records added successfully (if applicable)!
**Text #16 prompt #1 of 1**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo
Chaining class instance created
	Done creating prompt
	Sending request to gpt-3.5-turbo
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Completed
Processing 16_prompt00...
Original summaries DataFrame shape: (2, 19)
	Original summaries Dataframe columns: Index(['choice', 'timestamp', 'reference_id', 'article_title', 'text',
       'system_role', 'model', 'temperature', 'prep_step', 'summarize_task',
       'edit_task', 'simplify_task', 'simplify_audience', 'format_task',
       'full_summarize_task', 'folder', 'summary', 'headline',
       'simple_summary'],
      dtype='object')
Adding 2 rows to the database...
	Reference #16: Hypoh

# Models

In [3]:
openai_models(query='gpt-3.5')

gpt-3.5-turbo-16k-0613
gpt-3.5-turbo
gpt-3.5-turbo-0613-alpha-internal
gpt-3.5-turbo-0301
gpt-3.5-turbo-16k
gpt-3.5-turbo-0613-alpha-shared
gpt-3.5-turbo-0613


[<Model model id=gpt-3.5-turbo-16k-0613 at 0x1915f37c6b0> JSON: {
   "created": 1685474247,
   "id": "gpt-3.5-turbo-16k-0613",
   "object": "model",
   "owned_by": "openai",
   "parent": null,
   "permission": [
     {
       "allow_create_engine": false,
       "allow_fine_tuning": false,
       "allow_logprobs": true,
       "allow_sampling": true,
       "allow_search_indices": false,
       "allow_view": true,
       "created": 1690865619,
       "group": null,
       "id": "modelperm-0WUfVWoQgban6U6wF2YQVlq7",
       "is_blocking": false,
       "object": "model_permission",
       "organization": "*"
     }
   ],
   "root": "gpt-3.5-turbo-16k-0613"
 },
 <Model model id=gpt-3.5-turbo at 0x1915f37dc70> JSON: {
   "created": 1677610602,
   "id": "gpt-3.5-turbo",
   "object": "model",
   "owned_by": "openai",
   "parent": null,
   "permission": [
     {
       "allow_create_engine": false,
       "allow_fine_tuning": false,
       "allow_logprobs": true,
       "allow_sampling": true

# batch 7 gpt-3.5-turbo-0301

In [4]:
from functools import wraps
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\private")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from prompts import * # .py file stored in the path above
from db_orm import * 
from sources import *
from orm_summarize import *
from article_processing import *

@remote_sql_session
def get_from_queue(session, input_df, order_by='id', order='ASC'):
    """
    Return the matching records from the sources table as a pandas dataframe.

    Parameters:
    - input_df: A pandas DataFrame with the article records from the gpt_queue table or equivalent. Columns include 'title' and 'section'.
    - limit: The number of records to return.
    """
    def row_to_dict(row):
        result = session.query(Sources).filter_by(
            title=row['title'],
            section=row['section']
        ).limit(1).all()[0]
        
        sources_series = pd.Series({column.name: getattr(result, column.name) for column in result.__table__.columns})
        return sources_series

    sources_df = input_df.apply(row_to_dict, axis=1)
    ascending = True if order == 'ASC' else False
    sources_df.sort_values(order_by, ascending=ascending, inplace=True)
    return sources_df


#########
# Prep: Set parameters
folder_path = '../text/2023-08-01 discussion'
section = 'discussion'
local = False
n_choices = 3
article_limit = 1
temperature = 0.7
pause_per_request=0
# summary_iteration_id = iteration_id
iteration_id = 1
chatbot_id = iteration_id
# model = 'gpt-3.5-turbo-16k-0613'
# model = 'gpt-3.5-turbo'
model = 'gpt-3.5-turbo-0301'


# model = 'gpt-4'
save_outputs=False

def generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local, article_limit=article_limit):
    ### Set up
    qna_dict = dict()
    chatbot_dict = dict()
    references_df_dict = dict()

    # set the option to wrap text within cells
    pd.set_option('display.max_colwidth', 50)
    pd.set_option('display.max_rows', 20)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)

    ####### 
    # Step 1: Create sources table
    if local:
        text_df = parse_fulltext(folder_path, section).iloc[:article_limit if article_limit else len(text_df)]
    else:
        text_df = get_table(table='gpt_queue', limit=article_limit) # db_orm.py
    references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py

    ###### 
    # Step 2:  Add rows from gpt_queue table to sources table 
    bulk_append(table='sources', input_df=references_df_dict[iteration_id]) # db_orm.py

    # ##### 
    # Step 3: Get the new sources for summarization
    sources_df = get_from_queue(input_df=text_df, order_by='id', order='ASC')

    # ##### 
    # Step 4: Create summaries (functions contained in orm_summarize.py)
    chatbot_dict = batch_summarize( # orm_summarize.py
        sources_df, folder_path, prep_step, summarize_task, edit_task,  # parameter values found in prompts.py
        simplify_task, simplify_audience, format_task,
        chatbot_dict, temperature=temperature,
        system_role=system_role, model=model, max_tokens=800,
        n_choices=n_choices, pause_per_request=pause_per_request,
        iteration_id=iteration_id, save_outputs=save_outputs
        )
    #########
    # Step 5: Create summaries table
    qna_dict = create_summaries_df(
        qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id
        )

    ##########
    # Step 5: Add results to summaries and prompts table 
    bulk_append(table='summaries', input_df=qna_dict[iteration_id]) # db_orm.py

    return qna_dict[iteration_id]

if __name__ == "__main__":
    qna_dict = generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local=True, article_limit=article_limit)
    # print(qna_dict[iteration_id)
    qna_dict



Keys for text_dict: dict_keys([1])

Adding 1 rows to the database...
	** Already exists in the database: Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women.
New records added successfully (if applicable)!
**Text #16 prompt #1 of 1**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo-0301
Chaining class instance created
	Done creating prompt
	Sending request to gpt-3.5-turbo-0301
		Requesting 3 choices using gpt-3.5-turbo-0301
	Done sending request to GPT-3
	...Completed
Processing 16_prompt00...
Original summaries DataFrame shape: (3, 19)
	Original summaries Dataframe columns: Index(['choice', 'timestamp', 'reference_id', 'article_title', 'text',
       'system_role', 'model', 'temperature', 'prep_step', 'summarize_task',
       'edit_task', 'simplify_task', 'simplify_audience', 'format_task',
       'full_summarize_task', 'folder', 'summary', 'headline',
       'simple_summary'],
      dtype='object')
Adding 3 rows to the database...
	Refer

# batch 8: swap order of prep step and summarize task

In [2]:
from functools import wraps
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\private")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from prompts import * # .py file stored in the path above
from db_orm import * 
from sources import *
from orm_summarize import *
from article_processing import *

@remote_sql_session
def get_from_queue(session, input_df, order_by='id', order='ASC'):
    """
    Return the matching records from the sources table as a pandas dataframe.

    Parameters:
    - input_df: A pandas DataFrame with the article records from the gpt_queue table or equivalent. Columns include 'title' and 'section'.
    - limit: The number of records to return.
    """
    def row_to_dict(row):
        result = session.query(Sources).filter_by(
            title=row['title'],
            section=row['section']
        ).limit(1).all()[0]
        
        sources_series = pd.Series({column.name: getattr(result, column.name) for column in result.__table__.columns})
        return sources_series

    sources_df = input_df.apply(row_to_dict, axis=1)
    ascending = True if order == 'ASC' else False
    sources_df.sort_values(order_by, ascending=ascending, inplace=True)
    return sources_df


#########
# Prep: Set parameters
folder_path = '../text/2023-08-01 discussion'
section = 'discussion'
local = False
n_choices = 2
article_limit = 1
temperature = 0.7
pause_per_request=0
# summary_iteration_id = iteration_id
iteration_id = 1
chatbot_id = iteration_id
# model = 'gpt-3.5-turbo-16k-0613'
model = 'gpt-3.5-turbo'

# model = 'gpt-4'
save_outputs=False

def generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local, article_limit=article_limit):
    ### Set up
    qna_dict = dict()
    chatbot_dict = dict()
    references_df_dict = dict()

    # set the option to wrap text within cells
    pd.set_option('display.max_colwidth', 50)
    pd.set_option('display.max_rows', 20)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)

    ####### 
    # Step 1: Create sources table
    if local:
        text_df = parse_fulltext(folder_path, section).iloc[:article_limit if article_limit else len(text_df)]
    else:
        text_df = get_table(table='gpt_queue', limit=article_limit) # db_orm.py
    references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py

    ###### 
    # Step 2:  Add rows from gpt_queue table to sources table 
    bulk_append(table='sources', input_df=references_df_dict[iteration_id]) # db_orm.py

    # ##### 
    # Step 3: Get the new sources for summarization
    sources_df = get_from_queue(input_df=text_df, order_by='id', order='ASC')

    # ##### 
    # Step 4: Create summaries (functions contained in orm_summarize.py)
    chatbot_dict = batch_summarize( # orm_summarize.py
        sources_df, folder_path, prep_step, summarize_task, edit_task,  # parameter values found in prompts.py
        simplify_task, simplify_audience, format_task,
        chatbot_dict, temperature=temperature,
        system_role=system_role, model=model, max_tokens=800,
        n_choices=n_choices, pause_per_request=pause_per_request,
        iteration_id=iteration_id, save_outputs=save_outputs
        )
    #########
    # Step 5: Create summaries table
    qna_dict = create_summaries_df(
        qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id
        )

    ##########
    # Step 5: Add results to summaries and prompts table 
    bulk_append(table='summaries', input_df=qna_dict[iteration_id]) # db_orm.py

    return qna_dict[iteration_id]

if __name__ == "__main__":
    qna_dict = generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local=True, article_limit=article_limit)
    # print(qna_dict[iteration_id)
    qna_dict



Keys for text_dict: dict_keys([1])

Adding 1 rows to the database...
	** Already exists in the database: Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women.
New records added successfully (if applicable)!
**Text #16 prompt #1 of 1**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo
Chaining class instance created
	Done creating prompt
	Sending request to gpt-3.5-turbo
		Requesting 2 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Completed
Processing 16_prompt00...
Original summaries DataFrame shape: (2, 19)
	Original summaries Dataframe columns: Index(['choice', 'timestamp', 'reference_id', 'article_title', 'text',
       'system_role', 'model', 'temperature', 'prep_step', 'summarize_task',
       'edit_task', 'simplify_task', 'simplify_audience', 'format_task',
       'full_summarize_task', 'folder', 'summary', 'headline',
       'simple_summary'],
      dtype='object')
Adding 2 rows to the database...
	Reference #16: Hypoh

## use full text

In [4]:
from functools import wraps
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\private")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from prompts import * # .py file stored in the path above
from db_orm import * 
from sources import *
from orm_summarize import *
from article_processing import *

@remote_sql_session
def get_from_queue(session, input_df, order_by='id', order='ASC'):
    """
    Return the matching records from the sources table as a pandas dataframe.

    Parameters:
    - input_df: A pandas DataFrame with the article records from the gpt_queue table or equivalent. Columns include 'title' and 'section'.
    - limit: The number of records to return.
    """
    def row_to_dict(row):
        result = session.query(Sources).filter_by(
            title=row['title'],
            section=row['section']
        ).limit(1).all()[0]
        
        sources_series = pd.Series({column.name: getattr(result, column.name) for column in result.__table__.columns})
        return sources_series

    sources_df = input_df.apply(row_to_dict, axis=1)
    ascending = True if order == 'ASC' else False
    sources_df.sort_values(order_by, ascending=ascending, inplace=True)
    return sources_df


#########
# Prep: Set parameters
folder_path = '../text/2023-08-01'
# section = 'discussion'
section = None
local = False
n_choices = 4
article_limit = 1
temperature = 0.7
pause_per_request=0
# summary_iteration_id = iteration_id
iteration_id = 1
chatbot_id = iteration_id
model = 'gpt-3.5-turbo-16k-0613'
# model = 'gpt-3.5-turbo'
# model = 'gpt-3.5-turbo-0301'


# model = 'gpt-4'
save_outputs=False

def generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local, article_limit=article_limit):
    ### Set up
    qna_dict = dict()
    chatbot_dict = dict()
    references_df_dict = dict()

    # set the option to wrap text within cells
    pd.set_option('display.max_colwidth', 50)
    pd.set_option('display.max_rows', 20)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)

    ####### 
    # Step 1: Create sources table
    if local:
        text_df = parse_fulltext(folder_path, section).iloc[:article_limit if article_limit else len(text_df)]
    else:
        text_df = get_table(table='gpt_queue', limit=article_limit) # db_orm.py
    references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py

    ###### 
    # Step 2:  Add rows from gpt_queue table to sources table 
    bulk_append(table='sources', input_df=references_df_dict[iteration_id]) # db_orm.py

    # ##### 
    # Step 3: Get the new sources for summarization
    sources_df = get_from_queue(input_df=text_df, order_by='id', order='ASC')

    # ##### 
    # Step 4: Create summaries (functions contained in orm_summarize.py)
    chatbot_dict = batch_summarize( # orm_summarize.py
        sources_df, folder_path, prep_step, summarize_task, edit_task,  # parameter values found in prompts.py
        simplify_task, simplify_audience, format_task,
        chatbot_dict, temperature=temperature,
        system_role=system_role, model=model, max_tokens=800,
        n_choices=n_choices, pause_per_request=pause_per_request,
        iteration_id=iteration_id, save_outputs=save_outputs
        )
    #########
    # Step 5: Create summaries table
    qna_dict = create_summaries_df(
        qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id
        )

    ##########
    # Step 5: Add results to summaries and prompts table 
    bulk_append(table='summaries', input_df=qna_dict[iteration_id]) # db_orm.py

    return qna_dict[iteration_id]

if __name__ == "__main__":
    qna_dict = generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local=True, article_limit=article_limit)
    # print(qna_dict[iteration_id)
    qna_dict



Keys for text_dict: dict_keys([1])

Adding 1 rows to the database...
	** Already exists in the database: Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women.
New records added successfully (if applicable)!
**Text #5 prompt #1 of 1**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo-16k-0613
Chaining class instance created
	Done creating prompt
	Sending request to gpt-3.5-turbo-16k-0613
		Requesting 4 choices using gpt-3.5-turbo-16k-0613
	Done sending request to GPT-3
	...Completed
Processing 5_prompt00...
Original summaries DataFrame shape: (4, 19)
	Original summaries Dataframe columns: Index(['choice', 'timestamp', 'reference_id', 'article_title', 'text',
       'system_role', 'model', 'temperature', 'prep_step', 'summarize_task',
       'edit_task', 'simplify_task', 'simplify_audience', 'format_task',
       'full_summarize_task', 'folder', 'summary', 'headline',
       'simple_summary'],
      dtype='object')
Adding 4 rows to the database

# batch 9 add back simplify task

In [1]:
from functools import wraps
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\private")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from prompts import * # .py file stored in the path above
from db_orm import * 
from sources import *
from orm_summarize import *
from article_processing import *

@remote_sql_session
def get_from_queue(session, input_df, order_by='id', order='ASC'):
    """
    Return the matching records from the sources table as a pandas dataframe.

    Parameters:
    - input_df: A pandas DataFrame with the article records from the gpt_queue table or equivalent. Columns include 'title' and 'section'.
    - limit: The number of records to return.
    """
    def row_to_dict(row):
        result = session.query(Sources).filter_by(
            title=row['title'],
            section=row['section']
        ).limit(1).all()[0]
        
        sources_series = pd.Series({column.name: getattr(result, column.name) for column in result.__table__.columns})
        return sources_series

    sources_df = input_df.apply(row_to_dict, axis=1)
    ascending = True if order == 'ASC' else False
    sources_df.sort_values(order_by, ascending=ascending, inplace=True)
    return sources_df


#########
# Prep: Set parameters
folder_path = '../text/2023-08-01 discussion'
section = 'discussion'
local = False
n_choices = 4
article_limit = 1
temperature = 0.7
pause_per_request=0
# summary_iteration_id = iteration_id
iteration_id = 1
chatbot_id = iteration_id
model = 'gpt-3.5-turbo-16k-0613'
# model = 'gpt-3.5-turbo'

# model = 'gpt-4'
save_outputs=False

def generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local, article_limit=article_limit):
    ### Set up
    qna_dict = dict()
    chatbot_dict = dict()
    references_df_dict = dict()

    # set the option to wrap text within cells
    pd.set_option('display.max_colwidth', 50)
    pd.set_option('display.max_rows', 20)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)

    ####### 
    # Step 1: Create sources table
    if local:
        text_df = parse_fulltext(folder_path, section).iloc[:article_limit if article_limit else len(text_df)]
    else:
        text_df = get_table(table='gpt_queue', limit=article_limit) # db_orm.py
    references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py

    ###### 
    # Step 2:  Add rows from gpt_queue table to sources table 
    bulk_append(table='sources', input_df=references_df_dict[iteration_id]) # db_orm.py

    # ##### 
    # Step 3: Get the new sources for summarization
    sources_df = get_from_queue(input_df=text_df, order_by='id', order='ASC')

    # ##### 
    # Step 4: Create summaries (functions contained in orm_summarize.py)
    chatbot_dict = batch_summarize( # orm_summarize.py
        sources_df, folder_path, prep_step, summarize_task, edit_task,  # parameter values found in prompts.py
        simplify_task, simplify_audience, format_task,
        chatbot_dict, temperature=temperature,
        system_role=system_role, model=model, max_tokens=800,
        n_choices=n_choices, pause_per_request=pause_per_request,
        iteration_id=iteration_id, save_outputs=save_outputs
        )
    #########
    # Step 5: Create summaries table
    qna_dict = create_summaries_df(
        qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id
        )

    ##########
    # Step 5: Add results to summaries and prompts table 
    bulk_append(table='summaries', input_df=qna_dict[iteration_id]) # db_orm.py

    return qna_dict[iteration_id]

if __name__ == "__main__":
    qna_dict = generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local=True, article_limit=article_limit)
    # print(qna_dict[iteration_id)
    qna_dict



Keys for text_dict: dict_keys([1])

Adding 1 rows to the database...
	** Already exists in the database: Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women.
New records added successfully (if applicable)!
**Text #16 prompt #1 of 4**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo-16k-0613
Chaining class instance created
	Done creating prompt
	Sending request to gpt-3.5-turbo-16k-0613
		Requesting 4 choices using gpt-3.5-turbo-16k-0613
	Done sending request to GPT-3
	...Completed
**Text #16 prompt #2 of 4**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo-16k-0613
Chaining class instance created
	Done creating prompt
	Sending request to gpt-3.5-turbo-16k-0613
		Requesting 4 choices using gpt-3.5-turbo-16k-0613
	Done sending request to GPT-3
	...Completed
**Text #16 prompt #3 of 4**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo-16k-0613
Chaining class instance created
	Done creating prompt
	Sending request t

## gpt-3.5-turbo

In [2]:
from functools import wraps
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\private")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from prompts import * # .py file stored in the path above
from db_orm import * 
from sources import *
from orm_summarize import *
from article_processing import *

@remote_sql_session
def get_from_queue(session, input_df, order_by='id', order='ASC'):
    """
    Return the matching records from the sources table as a pandas dataframe.

    Parameters:
    - input_df: A pandas DataFrame with the article records from the gpt_queue table or equivalent. Columns include 'title' and 'section'.
    - limit: The number of records to return.
    """
    def row_to_dict(row):
        result = session.query(Sources).filter_by(
            title=row['title'],
            section=row['section']
        ).limit(1).all()[0]
        
        sources_series = pd.Series({column.name: getattr(result, column.name) for column in result.__table__.columns})
        return sources_series

    sources_df = input_df.apply(row_to_dict, axis=1)
    ascending = True if order == 'ASC' else False
    sources_df.sort_values(order_by, ascending=ascending, inplace=True)
    return sources_df


#########
# Prep: Set parameters
folder_path = '../text/2023-08-01 discussion'
section = 'discussion'
local = False
n_choices = 4
article_limit = 1
temperature = 0.7
pause_per_request=0
# summary_iteration_id = iteration_id
iteration_id = 1
chatbot_id = iteration_id
# model = 'gpt-3.5-turbo-16k-0613'
model = 'gpt-3.5-turbo'

# model = 'gpt-4'
save_outputs=False

def generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local, article_limit=article_limit):
    ### Set up
    qna_dict = dict()
    chatbot_dict = dict()
    references_df_dict = dict()

    # set the option to wrap text within cells
    pd.set_option('display.max_colwidth', 50)
    pd.set_option('display.max_rows', 20)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)

    ####### 
    # Step 1: Create sources table
    if local:
        text_df = parse_fulltext(folder_path, section).iloc[:article_limit if article_limit else len(text_df)]
    else:
        text_df = get_table(table='gpt_queue', limit=article_limit) # db_orm.py
    references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py

    ###### 
    # Step 2:  Add rows from gpt_queue table to sources table 
    bulk_append(table='sources', input_df=references_df_dict[iteration_id]) # db_orm.py

    # ##### 
    # Step 3: Get the new sources for summarization
    sources_df = get_from_queue(input_df=text_df, order_by='id', order='ASC')

    # ##### 
    # Step 4: Create summaries (functions contained in orm_summarize.py)
    chatbot_dict = batch_summarize( # orm_summarize.py
        sources_df, folder_path, prep_step, summarize_task, edit_task,  # parameter values found in prompts.py
        simplify_task, simplify_audience, format_task,
        chatbot_dict, temperature=temperature,
        system_role=system_role, model=model, max_tokens=800,
        n_choices=n_choices, pause_per_request=pause_per_request,
        iteration_id=iteration_id, save_outputs=save_outputs
        )
    #########
    # Step 5: Create summaries table
    qna_dict = create_summaries_df(
        qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id
        )

    ##########
    # Step 5: Add results to summaries and prompts table 
    bulk_append(table='summaries', input_df=qna_dict[iteration_id]) # db_orm.py

    return qna_dict[iteration_id]

if __name__ == "__main__":
    qna_dict = generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local=True, article_limit=article_limit)
    # print(qna_dict[iteration_id)
    qna_dict



Keys for text_dict: dict_keys([1])

Adding 1 rows to the database...
	** Already exists in the database: Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women.
New records added successfully (if applicable)!
**Text #16 prompt #1 of 4**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo
Chaining class instance created
	Done creating prompt
	Sending request to gpt-3.5-turbo
		Requesting 4 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Completed
**Text #16 prompt #2 of 4**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo
Chaining class instance created
	Done creating prompt
	Sending request to gpt-3.5-turbo
		Requesting 4 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Completed
**Text #16 prompt #3 of 4**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo
Chaining class instance created
	Done creating prompt
	Sending request to gpt-3.5-turbo
		Requesting 4 choices using gpt-3.5-turbo
	Don

## gpt-3.5-turbo-0301

In [3]:
from functools import wraps
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\private")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from prompts import * # .py file stored in the path above
from db_orm import * 
from sources import *
from orm_summarize import *
from article_processing import *

@remote_sql_session
def get_from_queue(session, input_df, order_by='id', order='ASC'):
    """
    Return the matching records from the sources table as a pandas dataframe.

    Parameters:
    - input_df: A pandas DataFrame with the article records from the gpt_queue table or equivalent. Columns include 'title' and 'section'.
    - limit: The number of records to return.
    """
    def row_to_dict(row):
        result = session.query(Sources).filter_by(
            title=row['title'],
            section=row['section']
        ).limit(1).all()[0]
        
        sources_series = pd.Series({column.name: getattr(result, column.name) for column in result.__table__.columns})
        return sources_series

    sources_df = input_df.apply(row_to_dict, axis=1)
    ascending = True if order == 'ASC' else False
    sources_df.sort_values(order_by, ascending=ascending, inplace=True)
    return sources_df


#########
# Prep: Set parameters
folder_path = '../text/2023-08-01 discussion'
section = 'discussion'
local = False
n_choices = 4
article_limit = 1
temperature = 0.7
pause_per_request=0
# summary_iteration_id = iteration_id
iteration_id = 1
chatbot_id = iteration_id
# model = 'gpt-3.5-turbo-16k-0613'
# model = 'gpt-3.5-turbo'
model = 'gpt-3.5-turbo-0301'

# model = 'gpt-4'
save_outputs=False

def generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local, article_limit=article_limit):
    ### Set up
    qna_dict = dict()
    chatbot_dict = dict()
    references_df_dict = dict()

    # set the option to wrap text within cells
    pd.set_option('display.max_colwidth', 50)
    pd.set_option('display.max_rows', 20)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)

    ####### 
    # Step 1: Create sources table
    if local:
        text_df = parse_fulltext(folder_path, section).iloc[:article_limit if article_limit else len(text_df)]
    else:
        text_df = get_table(table='gpt_queue', limit=article_limit) # db_orm.py
    references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py

    ###### 
    # Step 2:  Add rows from gpt_queue table to sources table 
    bulk_append(table='sources', input_df=references_df_dict[iteration_id]) # db_orm.py

    # ##### 
    # Step 3: Get the new sources for summarization
    sources_df = get_from_queue(input_df=text_df, order_by='id', order='ASC')

    # ##### 
    # Step 4: Create summaries (functions contained in orm_summarize.py)
    chatbot_dict = batch_summarize( # orm_summarize.py
        sources_df, folder_path, prep_step, summarize_task, edit_task,  # parameter values found in prompts.py
        simplify_task, simplify_audience, format_task,
        chatbot_dict, temperature=temperature,
        system_role=system_role, model=model, max_tokens=800,
        n_choices=n_choices, pause_per_request=pause_per_request,
        iteration_id=iteration_id, save_outputs=save_outputs
        )
    #########
    # Step 5: Create summaries table
    qna_dict = create_summaries_df(
        qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id
        )

    ##########
    # Step 5: Add results to summaries and prompts table 
    bulk_append(table='summaries', input_df=qna_dict[iteration_id]) # db_orm.py

    return qna_dict[iteration_id]

if __name__ == "__main__":
    qna_dict = generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local=True, article_limit=article_limit)
    # print(qna_dict[iteration_id)
    qna_dict



Keys for text_dict: dict_keys([1])

Adding 1 rows to the database...
	** Already exists in the database: Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women.
New records added successfully (if applicable)!
**Text #16 prompt #1 of 4**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo-0301
Chaining class instance created
	Done creating prompt
	Sending request to gpt-3.5-turbo-0301
		Requesting 4 choices using gpt-3.5-turbo-0301
	Done sending request to GPT-3
	...Completed
**Text #16 prompt #2 of 4**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo-0301
Chaining class instance created
	Done creating prompt
	Sending request to gpt-3.5-turbo-0301
		Requesting 4 choices using gpt-3.5-turbo-0301
	Done sending request to GPT-3
	...Completed
**Text #16 prompt #3 of 4**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo-0301
Chaining class instance created
	Done creating prompt
	Sending request to gpt-3.5-turbo-0301
		Reque

### debug prompts

In [ ]:
from functools import wraps
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\private")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from prompts import * # .py file stored in the path above
from db_orm import * 
from sources import *
from orm_summarize import *
from article_processing import *

@remote_sql_session
def get_from_queue(session, input_df, order_by='id', order='ASC'):
    """
    Return the matching records from the sources table as a pandas dataframe.

    Parameters:
    - input_df: A pandas DataFrame with the article records from the gpt_queue table or equivalent. Columns include 'title' and 'section'.
    - limit: The number of records to return.
    """
    def row_to_dict(row):
        result = session.query(Sources).filter_by(
            title=row['title'],
            section=row['section']
        ).limit(1).all()[0]
        
        sources_series = pd.Series({column.name: getattr(result, column.name) for column in result.__table__.columns})
        return sources_series

    sources_df = input_df.apply(row_to_dict, axis=1)
    ascending = True if order == 'ASC' else False
    sources_df.sort_values(order_by, ascending=ascending, inplace=True)
    return sources_df


#########
# Prep: Set parameters
folder_path = '../text/2023-08-01 discussion'
section = 'discussion'
local = False
n_choices = 4
article_limit = 1
temperature = 0.7
pause_per_request=0
# summary_iteration_id = iteration_id
iteration_id = 1
chatbot_id = iteration_id
# model = 'gpt-3.5-turbo-16k-0613'
# model = 'gpt-3.5-turbo'
model = 'gpt-3.5-turbo-0301'

# model = 'gpt-4'
save_outputs=False

def generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local, article_limit=article_limit):
    ### Set up
    qna_dict = dict()
    chatbot_dict = dict()
    references_df_dict = dict()

    # set the option to wrap text within cells
    pd.set_option('display.max_colwidth', 50)
    pd.set_option('display.max_rows', 20)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)

    ####### 
    # Step 1: Create sources table
    if local:
        text_df = parse_fulltext(folder_path, section).iloc[:article_limit if article_limit else len(text_df)]
    else:
        text_df = get_table(table='gpt_queue', limit=article_limit) # db_orm.py
    references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py

    ###### 
    # Step 2:  Add rows from gpt_queue table to sources table 
    bulk_append(table='sources', input_df=references_df_dict[iteration_id]) # db_orm.py

    # ##### 
    # Step 3: Get the new sources for summarization
    sources_df = get_from_queue(input_df=text_df, order_by='id', order='ASC')

    # ##### 
    # Step 4: Create summaries (functions contained in orm_summarize.py)
    chatbot_dict = batch_summarize( # orm_summarize.py
        sources_df, folder_path, prep_step, summarize_task, edit_task,  # parameter values found in prompts.py
        simplify_task, simplify_audience, format_task,
        chatbot_dict, temperature=temperature,
        system_role=system_role, model=model, max_tokens=800,
        n_choices=n_choices, pause_per_request=pause_per_request,
        iteration_id=iteration_id, save_outputs=save_outputs
        )
    #########
    # Step 5: Create summaries table
    qna_dict = create_summaries_df(
        qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id
        )

    ##########
    # Step 5: Add results to summaries and prompts table 
    bulk_append(table='summaries', input_df=qna_dict[iteration_id]) # db_orm.py

    return qna_dict[iteration_id]

if __name__ == "__main__":
    qna_dict = generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local=True, article_limit=article_limit)
    # print(qna_dict[iteration_id)
    qna_dict



Keys for text_dict: dict_keys([1])

Adding 1 rows to the database...
	** Already exists in the database: Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women.
New records added successfully (if applicable)!
**Text #16 prompt #1 of 4**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo-0301
Chaining class instance created
	Done creating prompt
	Sending request to gpt-3.5-turbo-0301
		Requesting 4 choices using gpt-3.5-turbo-0301
	Done sending request to GPT-3
	...Completed
**Text #16 prompt #2 of 4**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo-0301
Chaining class instance created
	Done creating prompt
	Sending request to gpt-3.5-turbo-0301
		Requesting 4 choices using gpt-3.5-turbo-0301
	Done sending request to GPT-3
	...Completed
**Text #16 prompt #3 of 4**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo-0301
Chaining class instance created
	Done creating prompt
	Sending request to gpt-3.5-turbo-0301
		Reque

# *End of Page*